# Preparing Libraries and Installations


In [1]:
%pip install bs4 selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/484.6 kB ? eta -:--:--
   ---------------------- ----------------- 276.5/484.6 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 484.6/484.6 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ---------------------------------------- 307.7/307.7 kB 9.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    StaleElementReferenceException,
)


from bs4 import BeautifulSoup

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

import pandas as pd
import time

# Scraping the Website

In [4]:
# URL Ulasan dari Female Daily
base_url = "https://reviews.femaledaily.com/products/moisturizer/sun-protection-1/emina/sun-protection-spf-30-pa?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=oldest&page={}"
page = 1

# Koneksi ke mongodb
username = "ayukrisna"
password = "bigdata"
host = "ayukrisna.c81wxan.mongodb.net"
dbname = "femaledaily_review"
uri = f"mongodb+srv://{username}:{password}@{host}/{dbname}?retryWrites=true&w=majority"

try:
    client = MongoClient(uri)
    db = client[dbname]
    collection = db['emina_review']

    print("Successfully connected to MongoDB!")
except ConnectionFailure as e:
    print(f"Failed to connect to MongoDB: {e}")
    exit()

try:
    # Inisialisasi option chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Inisialisasi WebDriver
    service = Service(executable_path = ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    #data frame
    df_data = pd.DataFrame(columns = ['age', 'skin_type', 'rating', 'recommended', 'review','usage_period', 'purchase_point'])

    # Scraping data
    while (page < 495):
        # Buka URL
        url = base_url.format(page)
        driver.get(url)
        print(page)
        wait = WebDriverWait(driver, 10)

        # Ekstrak dan print title
        # print(driver.title)
        time.sleep(5) #tunggu 5 detik

        # Get List Reviews
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        list_reviews = soup.find(class_='list-reviews')

        if not list_reviews:
            print("No reviews found on this page.")
            page += 1
            continue

        reviews = list_reviews.find_all(class_='review-card')
                       
        for review in reviews:
            #left column
            left_col = review.find(class_='review-card-col left')
            age_element = left_col.find('p', class_='profile-age')
            skin_type_element = left_col.find('p', class_='profile-description')

            # Extract age and skin type only if elements are found
            age = age_element.get_text(strip=True) if age_element else ""
            skin_type = skin_type_element.get_text(strip=True) if skin_type_element else ""

            #right column
            right_col = review.find(class_='review-card-col right')
            wrapper = right_col.find(class_='review-content-wrapper')

            #start ratings
            star_icons = wrapper.find_all('i', class_='icon-ic_big_star_full')
            rating = len(star_icons)

            #recommend
            recommendation_tag = wrapper.find('p', class_='recommend')
            if recommendation_tag:
                if 'recommends' in recommendation_tag.get_text():
                    recommended = "recommended"
                else:
                    recommended = "not recommended"
            else:
                recommended = ""

            #text review
            try:
                review_text_element = wrapper.find(class_='text-content')
                review_text = review_text_element.get_text(strip=True) if review_text_element else ""
            except (StaleElementReferenceException, NoSuchElementException):
                review_text = ""

            # Usage period and purchase point
            info_wrapper = wrapper.find(class_='information-wrapper')
            usage_period = ""
            purchase_point = ""
            if info_wrapper:
                usage_period_element = info_wrapper.find('span', string='Usage Period ')
                purchase_point_element = info_wrapper.find('span', string='Purchase Point ')

                # Extract usage period and purchase point only if elements are found
                if usage_period_element:
                    usage_period = usage_period_element.find_next('b').get_text(strip=True)
                if purchase_point_element:
                    purchase_point = purchase_point_element.find_next('b').get_text(strip=True)
            
            # simpan ke frame
            df_data.loc[len(df_data)] = [age, skin_type, rating, recommended, review_text, usage_period, purchase_point]

            #simpan ke mongodb
            review_data = {
                'age': age,
                'skin_type': skin_type,
                'rating': rating,
                'recommended': recommended,
                'review': review_text,
                'usage_period': usage_period,
                'purchase_point': purchase_point
            }
            collection.insert_one(review_data)
            print("Successfully inserted the data")

        # Memeriksa apakah ada button next page
        try:
            next_button = driver.find_element(By.ID, 'id_next_page')
            if 'paging-prev-text-inactive' in next_button.get_attribute('class'):
                break
        except NoSuchElementException:
            break

        page += 1
        time.sleep(2)

#exception
except WebDriverException as e:
    print(f"WebDriverException: {e}")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")
except TimeoutException:
    print("The element was not clickable within the given time")
finally:
    driver.quit()

pd.set_option('display.max_colwidth', None)
df_data

Successfully connected to MongoDB!
1
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
2
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
3
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
Successfully inserted the data
4
Successfully inserted the d

,age,skin_type,rating,recommended,review,usage_period,purchase_point
0,25 - 29,"Combination, Medium Dark, Warm",3,,"Beli ini online kebetulan pas banget baru tau Emina udah ngeluarin sunscreen jadi sekalian aja order, selain juga udah pensiun pake sunscreen skin aqua. Suka banget sama packagingnya, nggak tau kenapa kesannya imut-imut banget. Warnanya agak-agak kuning dan ada wanginya. Nggak ganggu sih tapi kalo aku pribadi kurang suka. Pas dipake nggak bikin wajah kerasa greasy dan cukup nyerap di aku. Jadi wajah nggak kelihatan kayak wajah bangun tidur. So far ini sunscreen yang paling aku suka dari sunscreen2 lain yg udah aku coba, tambah kemasannya super unyuuuu",,
1,25 - 29,"Combination, Medium Light, Neutral",5,,"Sun Protection ini harganya terbilang sangat murah, dibawah 30k tapi mampu melindungi kulit wajah dari matahari. Biasanya aku pakai pelembab atau bb cream yg ada spfnya, pergi siang-siang kulit bisa gosong, tapi ini enggak :D Cinta-lah sama Sun Protection ini, harga murah tp Alhamdulillah enggak ngebo'ong :D",,
2,19 - 24,"Oily, Medium, Warm",5,,"Sukaa banget sama produk ini, warna krim nya kuning gitu terus wanginya lumayan enak teksturnya aku suka banget mudah di apply dan cepet banget meresapnya. Harganya murah aku beli 25k, kemasannya cute tapi rada ga enak kl dipegang lama, overall aku cinta banget sama produk ini, ini lebih enak daripada sunblock2 ku terdahulu",,
3,25 - 29,"Oily, Medium, Warm",4,,"Beli ini krn Skin Aqua yg aku pake habis, dan susahhh banget mau beli yg baru. Pas liat ini di etalase, akhirnya beli deh. Awalnya aku smpt ragu krn kata mbak2 yg jual ini krim gitu, krn muka aku acne prone dan kebiasa pakai SA yg gel-based. Ternyata aku malah suka banget pake ini. Meskipun krim tapi ringan dan lumayan cepat meresap, ngga lengket. Nggak bikin muka keliatan lusuh dan berminyak juga loh kl udh bbrp jam, pdhl cm pake produk ini+loose powder. Biasanya pk SA+loose powder muka aku udh kucel bgt, jd amazed bgt pas nyoba ini. Harganya juga terjangkau cm sekitar 30rb. Yg aku ga suka produk ini baunya agak aneh menurutku",,
4,19 - 24,"Combination, Medium Light, Warm",3,,"sun protection ini cukup murah untuk isi yang luamyan banyak..gampang nyerepnya, jadi bisa ditumpuk sama bb/foundie..cuma kayanya kurang cocok kalo buat yang mukanya berminyak, makin parah jadinya.. tapi tergantung cuaca juga sih, kl udaranya lagi ga bagus dan bikin kering, sunprotection ini bantuu banget..",,
...,...,...,...,...,...,...,...
4933,19 - 24,"Normal, Medium Dark, Neutral",1,,Dulu pernah pake dan ga cocok pake sunscreen ini bikin bruntusan trs dipake jd berminyak gitu krna ga meresap2. Bau nya juga ga suka aneh gitu. Baru pakai 2x udh ngerasa ga cocok jd ak kasihin org lain.,,
4934,18 and Under,"Combination, Light, Neutral",5,,ini sunscreen indo yang bagus banget ga bikin white cast di muka ku dan juga ga bikin berminyak harganya juga terjangkau trus baunya juga enak lagi bau aloe vera suka banget pokoknya . repurchase maybe pengen coba2 yang lain dulu,,
4935,19 - 24,"Oily, Medium Light, Warm",2,,dia sunscreen keduaku setelah parasol. sunscreen emina ini warnanya kuning pucat berbentuk krim. entah kenapa di aku awalnya bagus banget eh tapi lama-kelamaan aku ngerasa ini tuh buat muka aku kusam dan lebih berminyak. jadinya aku stop pemakaian dan berpindah ke skinaqua,,
4936,18 and Under,"Dry, Medium Light, Neutral",5,,"sunscreen favorit! sering bgt aku pake buat ke sekolah, karna teksturnya ringan dan ga lengket. warna nya kuning pucat gitu wkw, dan buat wanginya aku gatau wangi apa.. tapi ga mengganggu ko:) ini cocok bgt sih buat anak sekolahan, karna teksturnya yg ringan. dan buat harga, ini ga mahal2 amat ko✨. Repurchase? Yes",,


# Delete Duplicate

In [6]:
all_data = list(collection.find())
df_data = pd.DataFrame(all_data)
df_data

,_id,age,skin_type,rating,recommended,review,usage_period,purchase_point
0,6649cafafaa7c259d1c4dc04,19 - 24,"Combination, Medium Light, Cool",2,not recommended,"Aku ga terlalu cocok pake produk ini terlebih lagi muka aku oily skin jadi dipake 2 kali langsungkeluar jerawatnya, mungkin untuk kulit yang dryskin bisa di coba tapi aku cukup sekali pake produk ini karena kalo ga cocok bikin kulit jerawat besar²",Less than 1 week,Shopee
1,6649cafbfaa7c259d1c4dc05,19 - 24,"Combination, Medium, Neutral",3,not recommended,"Sunscreen pertama aku sewaktu kenal skincare, first apply aku kurang suka sama fragrance nya sangat kencang, dia teksturnya gel gitu dan gampang di apply di kulit. Dan kalian tau aku udah 4x repurchase karna harganya yang affordable sewaktu aku masih sekolah. Untuk hasil dari pemakaian ntah kenapa aku kyk ngerasa keringettan gitu ya huhu",3 months - 6 months,Shopee
2,6649cafbfaa7c259d1c4dc06,30 - 34,"Combination, Medium Light, Cool",3,recommended,"Teksturnya agak watery gituDiblendnya agak lumayan usahaDi muka saya, kerasa panas pas dipakai. Dan ada aroma agak tajemnya. Tapi emang harganya sangat affordable. Kalau pas keringetan, lumayan keliatan putih putih gitu bulir keringetnya dan agak pedih dimata.",1 week - 1 month,Super Indo
3,6649cafbfaa7c259d1c4dc07,19 - 24,"Oily, Medium, Neutral",5,recommended,"udah pake ini dari jaman SMA hahah seneng banget bisa cocok. suka banget sama texturenya yang ringan, gampang nyerep, gampang diblend, gak bikin gerah, gak ada whitecast. yang bikin seneng lagi, harganya murah meriah dan gampang didapetin.",More than 1 year,Alfamart
4,6649cafbfaa7c259d1c4dc08,19 - 24,"Normal, Medium, Neutral",3,not recommended,"ini adalah suncreen pertamaku , dulu cocok banget krn harganya sangat friendly di kantong pelajar , mudah untuk di apply juga .. cuma gatau kenapa tiap aku habis pakai ini muka jadi keliatan lebih kusam , jd aku ga terlalu merekomendasikannya siii",6 months - 1 year,Guardian
...,...,...,...,...,...,...,...,...
14891,664a1e1cd0780c70806093dd,19 - 24,"Normal, Medium Dark, Neutral",1,,Dulu pernah pake dan ga cocok pake sunscreen ini bikin bruntusan trs dipake jd berminyak gitu krna ga meresap2. Bau nya juga ga suka aneh gitu. Baru pakai 2x udh ngerasa ga cocok jd ak kasihin org lain.,,
14892,664a1e1cd0780c70806093de,18 and Under,"Combination, Light, Neutral",5,,ini sunscreen indo yang bagus banget ga bikin white cast di muka ku dan juga ga bikin berminyak harganya juga terjangkau trus baunya juga enak lagi bau aloe vera suka banget pokoknya . repurchase maybe pengen coba2 yang lain dulu,,
14893,664a1e1cd0780c70806093df,19 - 24,"Oily, Medium Light, Warm",2,,dia sunscreen keduaku setelah parasol. sunscreen emina ini warnanya kuning pucat berbentuk krim. entah kenapa di aku awalnya bagus banget eh tapi lama-kelamaan aku ngerasa ini tuh buat muka aku kusam dan lebih berminyak. jadinya aku stop pemakaian dan berpindah ke skinaqua,,
14894,664a1e1cd0780c70806093e0,18 and Under,"Dry, Medium Light, Neutral",5,,"sunscreen favorit! sering bgt aku pake buat ke sekolah, karna teksturnya ringan dan ga lengket. warna nya kuning pucat gitu wkw, dan buat wanginya aku gatau wangi apa.. tapi ga mengganggu ko:) ini cocok bgt sih buat anak sekolahan, karna teksturnya yg ringan. dan buat harga, ini ga mahal2 amat ko✨. Repurchase? Yes",,


In [7]:
df_data.drop_duplicates(subset=['age', 'skin_type', 'rating', 'recommended', 'review', 'usage_period', 'purchase_point'], keep='first', inplace=True)
df_data

,_id,age,skin_type,rating,recommended,review,usage_period,purchase_point
0,6649cafafaa7c259d1c4dc04,19 - 24,"Combination, Medium Light, Cool",2,not recommended,"Aku ga terlalu cocok pake produk ini terlebih lagi muka aku oily skin jadi dipake 2 kali langsungkeluar jerawatnya, mungkin untuk kulit yang dryskin bisa di coba tapi aku cukup sekali pake produk ini karena kalo ga cocok bikin kulit jerawat besar²",Less than 1 week,Shopee
1,6649cafbfaa7c259d1c4dc05,19 - 24,"Combination, Medium, Neutral",3,not recommended,"Sunscreen pertama aku sewaktu kenal skincare, first apply aku kurang suka sama fragrance nya sangat kencang, dia teksturnya gel gitu dan gampang di apply di kulit. Dan kalian tau aku udah 4x repurchase karna harganya yang affordable sewaktu aku masih sekolah. Untuk hasil dari pemakaian ntah kenapa aku kyk ngerasa keringettan gitu ya huhu",3 months - 6 months,Shopee
2,6649cafbfaa7c259d1c4dc06,30 - 34,"Combination, Medium Light, Cool",3,recommended,"Teksturnya agak watery gituDiblendnya agak lumayan usahaDi muka saya, kerasa panas pas dipakai. Dan ada aroma agak tajemnya. Tapi emang harganya sangat affordable. Kalau pas keringetan, lumayan keliatan putih putih gitu bulir keringetnya dan agak pedih dimata.",1 week - 1 month,Super Indo
3,6649cafbfaa7c259d1c4dc07,19 - 24,"Oily, Medium, Neutral",5,recommended,"udah pake ini dari jaman SMA hahah seneng banget bisa cocok. suka banget sama texturenya yang ringan, gampang nyerep, gampang diblend, gak bikin gerah, gak ada whitecast. yang bikin seneng lagi, harganya murah meriah dan gampang didapetin.",More than 1 year,Alfamart
4,6649cafbfaa7c259d1c4dc08,19 - 24,"Normal, Medium, Neutral",3,not recommended,"ini adalah suncreen pertamaku , dulu cocok banget krn harganya sangat friendly di kantong pelajar , mudah untuk di apply juga .. cuma gatau kenapa tiap aku habis pakai ini muka jadi keliatan lebih kusam , jd aku ga terlalu merekomendasikannya siii",6 months - 1 year,Guardian
...,...,...,...,...,...,...,...,...
14891,664a1e1cd0780c70806093dd,19 - 24,"Normal, Medium Dark, Neutral",1,,Dulu pernah pake dan ga cocok pake sunscreen ini bikin bruntusan trs dipake jd berminyak gitu krna ga meresap2. Bau nya juga ga suka aneh gitu. Baru pakai 2x udh ngerasa ga cocok jd ak kasihin org lain.,,
14892,664a1e1cd0780c70806093de,18 and Under,"Combination, Light, Neutral",5,,ini sunscreen indo yang bagus banget ga bikin white cast di muka ku dan juga ga bikin berminyak harganya juga terjangkau trus baunya juga enak lagi bau aloe vera suka banget pokoknya . repurchase maybe pengen coba2 yang lain dulu,,
14893,664a1e1cd0780c70806093df,19 - 24,"Oily, Medium Light, Warm",2,,dia sunscreen keduaku setelah parasol. sunscreen emina ini warnanya kuning pucat berbentuk krim. entah kenapa di aku awalnya bagus banget eh tapi lama-kelamaan aku ngerasa ini tuh buat muka aku kusam dan lebih berminyak. jadinya aku stop pemakaian dan berpindah ke skinaqua,,
14894,664a1e1cd0780c70806093e0,18 and Under,"Dry, Medium Light, Neutral",5,,"sunscreen favorit! sering bgt aku pake buat ke sekolah, karna teksturnya ringan dan ga lengket. warna nya kuning pucat gitu wkw, dan buat wanginya aku gatau wangi apa.. tapi ga mengganggu ko:) ini cocok bgt sih buat anak sekolahan, karna teksturnya yg ringan. dan buat harga, ini ga mahal2 amat ko✨. Repurchase? Yes",,


In [8]:
collection = db['emina_review_deleted_duplicates']
collection.insert_many(df_data.to_dict('records'))

InsertManyResult([ObjectId('6649cafafaa7c259d1c4dc04'), ObjectId('6649cafbfaa7c259d1c4dc05'), ObjectId('6649cafbfaa7c259d1c4dc06'), ObjectId('6649cafbfaa7c259d1c4dc07'), ObjectId('6649cafbfaa7c259d1c4dc08'), ObjectId('6649cafbfaa7c259d1c4dc09'), ObjectId('6649cafbfaa7c259d1c4dc0a'), ObjectId('6649cafbfaa7c259d1c4dc0b'), ObjectId('6649cafbfaa7c259d1c4dc0c'), ObjectId('6649cafbfaa7c259d1c4dc0d'), ObjectId('6649cb06faa7c259d1c4dc0e'), ObjectId('6649cb06faa7c259d1c4dc0f'), ObjectId('6649cb06faa7c259d1c4dc10'), ObjectId('6649cb06faa7c259d1c4dc11'), ObjectId('6649cb06faa7c259d1c4dc12'), ObjectId('6649cb06faa7c259d1c4dc13'), ObjectId('6649cb06faa7c259d1c4dc14'), ObjectId('6649cb06faa7c259d1c4dc15'), ObjectId('6649cb06faa7c259d1c4dc16'), ObjectId('6649cb06faa7c259d1c4dc17'), ObjectId('6649cb10faa7c259d1c4dc18'), ObjectId('6649cb10faa7c259d1c4dc19'), ObjectId('6649cb10faa7c259d1c4dc1a'), ObjectId('6649cb10faa7c259d1c4dc1b'), ObjectId('6649cb10faa7c259d1c4dc1c'), ObjectId('6649cb10faa7c259d1c4dc